In [1]:
import pandas as pd
import numpy as np
import boto3
from  tqdm import tqdm

In [2]:
metadata_path = '~/Dropbox/projects/autolocal/data/index/metadata.csv'
metadata = pd.read_csv(metadata_path).drop(columns=['Unnamed: 0.1'])
metadata = metadata.rename(columns={'Unnamed: 0': 'doc_id'})

In [3]:
metadata.head()

,doc_id,city,committee,date,doc_type,meeting_type,url,local_path_pdf,local_path_txt,doc_format
0,Cupertino_2019-10-29_Legislative-Review-Commit...,Cupertino,Legislative Review Committee,2019-10-29,Agenda,NaN,https://cupertino.legistar.com/View.ashx?M=A&I...,../data/docs/cupertino/Cupertino_2019-10-29_Le...,../data/docs/cupertino/Cupertino_2019-10-29_Le...,pdf
1,Cupertino_2019-10-28_Planning-Commission_Agenda,Cupertino,Planning Commission,2019-10-28,Agenda,NaN,https://cupertino.legistar.com/View.ashx?M=A&I...,../data/docs/cupertino/Cupertino_2019-10-28_Pl...,../data/docs/cupertino/Cupertino_2019-10-28_Pl...,pdf
2,Cupertino_2019-10-24_Administrative-Hearing_Ag...,Cupertino,Administrative Hearing,2019-10-24,Agenda,NaN,https://cupertino.legistar.com/View.ashx?M=A&I...,../data/docs/cupertino/Cupertino_2019-10-24_Ad...,../data/docs/cupertino/Cupertino_2019-10-24_Ad...,pdf
3,Cupertino_2019-10-24_Sustainability-Commission...,Cupertino,Sustainability Commission,2019-10-24,Agenda,NaN,https://cupertino.legistar.com/View.ashx?M=A&I...,../data/docs/cupertino/Cupertino_2019-10-24_Su...,../data/docs/cupertino/Cupertino_2019-10-24_Su...,pdf
4,Cupertino_2019-10-23_Teen-Commission_Agenda,Cupertino,Teen Commission,2019-10-23,Agenda,NaN,https://cupertino.legistar.com/View.ashx?M=A&I...,../data/docs/cupertino/Cupertino_2019-10-23_Te...,../data/docs/cupertino/Cupertino_2019-10-23_Te...,pdf


In [9]:
dynamodb = boto3.resource('dynamodb', endpoint_url='http://localhost:8000')
table = dynamodb.Table('autolocal-documents')

In [10]:
table_args = {
    'TableName': 'autolocal-documents',
    'KeySchema': [
        {
            'AttributeName': 'doc_id',
            'KeyType': 'HASH'
        }
    ],
    'AttributeDefinitions': [
        {
            'AttributeName': 'doc_id',
            'AttributeType': 'S'
        }        
    ],
    'ProvisionedThroughput':
    {
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
}
table = dynamodb.create_table(**table_args)

In [11]:
def row2item(row):
    row = dict(row)
    item = {}
    for k,v in row.items():
        if k in ['local_path_pdf', 'local_path_txt']:
            v = v[8:]
        if isinstance(v, str):
            item[k] = v
        elif np.isnan(v):
            pass
    return item

In [12]:
with table.batch_writer() as batch:
    for i, row in tqdm(metadata.iterrows()):
        item = row2item(row)
        batch.put_item(Item=item)

31678it [01:09, 456.50it/s]


In [62]:
s = '../data/docs/cupertino/Cupertino_2019-10-29_Legislative-Review-Committee_Agenda.txt'
s[8:]

'docs/cupertino/Cupertino_2019-10-29_Legislative-Review-Committee_Agenda.txt'

In [40]:
with table.batch_writer() as batch:
    for 

SyntaxError: invalid syntax (<ipython-input-40-e806202a7fbb>, line 1)